In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, ImageMessage, TextSendMessage
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")
app_1 = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )
# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

準備QuickReply的Button


'''
from linebot.models import (
    PostbackEvent,QuickReply, QuickReplyButton,URIAction,
    CameraAction, CameraRollAction,MessageAction
)
## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
action=MessageAction(
    label="確認結果", 
    text="確認結果"
    )
)
## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
action=CameraAction(label="拍照")
)


## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
action=CameraRollAction(label="上傳照片")
)
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, cameraQuickReplyButton, cameraRollQRB]
)
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quickReplyTextSendMessage = TextSendMessage(text='請您選擇拍照或上傳照片', quick_reply=quickReplyList)



In [5]:
"""
建立flextemplate
"""
def generate_flextemplate_dict(web_pic,shoes_name):
    flextemplate = {
      "type": "flex",
      "altText": "Flex Message",
      "contents": {
        "type": "bubble",
        "direction": "ltr",
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "請您對這雙鞋做評價"
            },
            {
              "type": "image",
              "url": web_pic,
              "align": "start",
              "size": "full",
              "aspectRatio": "1.91:1"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "評5分",
                "text": "評價五分",
                "data": "tag=5&product="+shoes_name+"&folder=商品推薦"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "評4分",
                "text": "評價四分",
                "data": "tag=4&product="+shoes_name+"&folder=商品推薦"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "評3分",
                "text": "評價三分",
                "data": "tag=3&product="+shoes_name+"&folder=商品推薦"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "評2分",
                "text": "評價二分",
                "data": "tag=2&product="+shoes_name+"&folder=商品推薦"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "評1分",
                "text": "評價一分",
                "data": "tag=1&product="+shoes_name+"&folder=商品推薦"
              },
              "flex": 0
            }
          ]
        }
      }
    }
    return flextemplate

## career_flextemplate_dict==>changing

In [ ]:
"""
建立flextemplate
"""
def career_flextemplate_dict(web_pic,shoes_name):
    flextemplate = {
      "type": "flex",
      "altText": "Flex Message",
      "contents": {
        "type": "bubble",
        "direction": "ltr",
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "問題",
              "weight": "bold"
            },
            {
              "type": "text",
              "text": "請問您是下列何種職業?",
              "align": "start",
              "weight": "regular",
              "wrap": true
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "自由業",
                "text": "自由業",
                "data": "tag=Freelance&folder=rich_menu_4"
              },
              "flex": 3,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "服務業",
                "text": "服務業",
                "data": "tag=services&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "金融證券業",
                "text": "金融證券業",
                "data": "tag=inancial_securities&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "製造業",
                "text": "製造業",
                "data": "tag=manufacturing&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "國營事業/公教人員",
                "text": "國營事業/公教人員",
                "data": "tag=state-ownedorpublic_education&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "電子業",
                "text": "電子業",
                "data": "tag=electronics&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "學生",
                "text": "學生",
                "data": "tag=students&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "資訊業",
                "text": "資訊業",
                "data": "tag=information_industry&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "家管",
                "text": "家管",
                "data": "tag=home_management&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "醫務人員",
                "text": "醫務人員",
                "data": "tag=medical_staff&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "其他行業",
                "text": "其他行業",
                "data": "tag=other_industries&folder=rich_menu_4"
              },
              "flex": 2,
              "color": "#070210",
              "height": "sm",
              "style": "link"
            }
          ]
        }
      }
    }
    return flextemplate

In [6]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    ############################關注or問題一
    replyJsonPath = "素材/問題一/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = {
      "type": "text",
      "text": "哈囉!" + str(user_profile.display_name) + \
              "\n歡迎您使用「履視步爽」\n目前我們的服務完全免費"+u"\U0001f600"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
    
        
    #綁定 rich_menu
    linkRichMenuId = open("素材/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)


    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )
    
    #擷取tag訊息
    dn = "ID_message/user_event.txt"
    user_event = {}
    with open(dn, "a") as myfile:
        user_event["user_id"] = vars(user_profile)["user_id"]
        user_event["tag"] = "user of first entering"
        user_event["time"] = event.timestamp
        user_event["product_ID"] = "now to choose"
        myfile.write(json.dumps(user_event,sort_keys=True))
        myfile.write('\r\n')

## 文字訊息

In [7]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

譯json後，將消息回傳給用戶

無則不發送訊息

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage, PostbackEvent
)
from os import listdir
from urllib.parse import parse_qs

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    
    # 指定要列出素材的目錄
    mypath = "素材/"
    # 取得所有檔案與子目錄名稱
    list_text = listdir(mypath)
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
    user_profile = line_bot_api.get_profile(event.source.user_id)


    if event.message.text in list_text:
        
        replyJsonPath = "素材/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = event.message.text
            user_event["time"] = event.timestamp
            user_event["product_ID"] = "now to choose"
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
    
    elif '確認結果' == event.message.text:
        # 讀取資料夾內.jpg檔案的數量
        pwd = '/root/darknet/result_img'
        imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        
        # 如果測試照片還沒輸出，回覆使用者稍後確認
        if len(imgWaitToTest) == 0:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='測試中，請稍後再確認，謝謝。')
            )
        
        # 如果測試照片已經輸出，則複製照片到對外IP的資料夾內，等待發送給使用者
        elif len(imgWaitToTest) != 0:
            CopyFileWaitToPost(event.source.user_id, event.message.id)
            
            # 準備輸出多張照片的list
            result_message_array = []
            while True:
                # 檢查照片是否已經到到對外IP的資料夾內部
                pwd = '/root/linechatbot/素材/user/'+event.source.user_id
                if os.path.exists(pwd):
                    imgWaitToPost = [i for i in os.listdir(pwd) if '.jpg' in i]
                if len(imgWaitToPost) !=0 :
                    for img in imgWaitToPost:
                        url = 'https://sneaker.serveo.net/%E7%B4%A0%E6%9D%90/user/'+event.source.user_id+'/'+img
                        print(url)
                        String = """ImageSendMessage(
                                     original_content_url="{}",
                                     preview_image_url="{}"
                                 )""".format(url, url)
                        result_message_array.append(eval(String))
                    line_bot_api.reply_message(
                        event.reply_token,
                        result_message_array
                    )
                    break
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='請上傳照片')
            )
    
    elif '圖片找鞋' == event.message.text:
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
    )
            
    
    elif event.message.text in os.listdir('./素材/'):
        # 讀取本地檔案，並轉譯成消息
        result_message_array =[]
        replyJsonPath = "素材/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    else:
        pass
        # 發送
#         line_bot_api.reply_message(
#             event.reply_token,
#             TextSendMessage(text='很抱歉，我不了解你的需求。')
#         )


## Postback Event

In [8]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送
    若folder為商品推薦，則
        開始評分
        在判斷評分次數是否小於3
        是，則
            發送評分訊息
        否，則
            發送文字訊息
        擷取user_score_event訊息

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送
擷取tag訊息

'''
from linebot.models import (
    PostbackEvent,QuickReply, QuickReplyButton,URIAction,
    CameraAction, CameraRollAction,MessageAction
)

from urllib.parse import parse_qs 
import sys, random, datetime, time, json
from kafka import KafkaProducer
import numpy as np
import pandas as pd

count = 0
@handler.add(PostbackEvent)
def process_postback_event(event):
    
    #設定要連線到Kafka集群的相關設定, 並產生一個Kafka的Producer的實例
    producer1 = KafkaProducer(
        # 指定Kafka集群伺服器(["docker.compose.host.name:9092"])
        bootstrap_servers = ["34.80.160.58:9092"],
        # 指定msgKey的序列化器, 若Key為None, 無法序列化, 透過producer直接給值
        # 指定msgValue的序列化器
        value_serializer = lambda m: json.dumps(m).encode('ascii'),
    )
        
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        #讀取其reply.json，轉譯成消息
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
         
        top_line = {
          "type": "text",
          "text": "已記下，放在心上"
        }
        result_message_array.insert(0, TextSendMessage.new_from_json_dict(top_line))
        
        ###############################
        if query_string_dict.get('folder')[0] == '商品推薦':
            global count
            if count < 3:
                flextemplate_list =[]
                df = pd.read_csv('database/all_shoe_color_cat_en.csv', encoding="utf-8")
                df_shoes = df[["pic_url","pid"]]#EE6461
                df_shoes_sample = df_shoes.sample(1)
                shoes_np = np.array(df_shoes_sample)#np.ndarray()
                list_shoes=shoes_np.tolist()[0]#list
                number = len(list_shoes)
                pic_url = list_shoes[0]
                name_shoes = list_shoes[1]
                
                flextemplate_dict = generate_flextemplate_dict(pic_url,name_shoes)
                ##發送訊息
                line_bot_api.reply_message(
                    event.reply_token,
                    FlexSendMessage.new_from_json_dict(flextemplate_dict))
                count += 1                        
                if query_string_dict.get('tag')[0] == 'Productsrecommend':
                    #擷取tag訊息
                    dn = "ID_message/user_menu_event.txt"
                    user_event = {}
                    with open(dn, "a") as myfile:
                        user_event["user_id"] = vars(user_profile)["user_id"]
                        user_event["tag"] = query_string_dict.get('tag')[0]
                        user_event["time"] = event.timestamp
                        myfile.write(json.dumps(user_event,sort_keys=True))
                        myfile.write('\r\n')
                    
                    # 產生要發佈到Kafka的訊息
                    future1 = producer1.send(topic="kibana_test1", value=user_event)
                    record_metadata1 = future1.get(timeout=10)
                    print("Message sending completed!")
                    time.sleep(random.randint(1,5))
                else:
                    #擷取tag訊息
                    dn = "ID_message/user_score_event.txt"
                    user_event = {}
                    with open(dn, "a") as myfile:
                        user_event["user_id"] = vars(user_profile)["user_id"]
                        user_event["score"] = query_string_dict.get('tag')[0]
                        user_event["product_ID"] = query_string_dict.get('product')[0]
                        user_event["time"] = event.timestamp
                        myfile.write(json.dumps(user_event,sort_keys=True))
                        myfile.write('\r\n')
#                         # 產生要發佈到Kafka的訊息
#                         future1 = producer1.send(topic="kibana_test", value=user_event)
#                         record_metadata1 = future1.get(timeout=10)
#                         print("Message sending completed!")
#                         time.sleep(random.randint(1,5))
            
            else:
                count = 0
                result_template = []
                userId = random.randint(1,610)
                print(userId)
                input_tuple = from_mysql_table_userId_rating_to_shoesID_list("TEST2", userId)
                # 連接 MySQL 資料庫
                list_col_color=[]
                list_shoes=from_mysql_shoesID_to_line_chat_bot_more("TEST2", input_tuple)
                list_shoes[:6]
            #     print(list_shoes)
                num = len(list_shoes[:6])
#                 print(num)
                for i in range(0, num):
                    #輸入值
                    #main_color, web_shoes, web_pic, name_shoes = list_shoes[i]
                    productID = list_shoes[i][0][0]
                    catatory = list_shoes[i][0][1]
                    web_shoes = list_shoes[i][0][2]
                    mcolor = list_shoes[i][0][3]
                    scolor = list_shoes[i][0][4]
                    web_pic = list_shoes[i][0][5]
                    Brand = list_shoes[i][0][6]
                    name_shoes= list_shoes[i][0][7]

                    #循環加到 list_col_color
                    col_1 = {
                            "thumbnailImageUrl": web_pic,
                            "title": Brand+" | "+productID,
                            "text": name_shoes,
                            "actions": [
                              {
                                "type": "postback",
                                "label": "主色系:"+mcolor,
                                "text": web_shoes,
                                "data": "tag="+productID
                              }
                            ]
                          }
                    list_col_color.append(col_1)

                #輸出到linebot輪播
                data ={
                    "type": "template",
                    "altText": "this is a carousel template",
                    "template":
                    {
                        "type": "carousel",
                        "actions": [],
                        "columns": list_col_color
                    }
                    }
#                 print(data)
                hello = {
                  "type": "text",
                  "text": "感謝您的回覆資訊，將提供專屬於您的個人推薦清單"
                }
                result_template.insert(1,TemplateSendMessage.new_from_json_dict(data))
                result_template.insert(0, TextSendMessage.new_from_json_dict(hello))
                
                line_bot_api.reply_message(
                    event.reply_token,
                    result_template)
                #擷取tag訊息
                dn = "ID_message/user_score_event.txt"
                user_event = {}
                with open(dn, "a") as myfile:
                    user_event["user_id"] = vars(user_profile)["user_id"]
                    user_event["score"] = query_string_dict.get('tag')[0]
                    user_event["product_ID"] = query_string_dict.get('product')[0]
                    user_event["time"] = event.timestamp
                    myfile.write(json.dumps(user_event,sort_keys=True))
                    myfile.write('\r\n')
#                     # 產生要發佈到Kafka的訊息
#                     future1 = producer1.send(topic="kibana_test", value=user_event)
#                     record_metadata1 = future1.get(timeout=10)
#                     print("Message sending completed!")
#                     time.sleep(random.randint(1,5))
                
        elif query_string_dict.get('folder')[0] == '顏色找鞋':
            line_bot_api.reply_message(
                event.reply_token,
                quickReplyTextSendMessage
            )
            #擷取tag訊息
            dn = "ID_message/user_menu_event.txt"
            user_event = {}
            with open(dn, "a") as myfile:
                user_event["user_id"] = vars(user_profile)["user_id"]
                user_event["menu"] = query_string_dict.get('tag')[0]
                user_event["time"] = event.timestamp
                myfile.write(json.dumps(user_event,sort_keys=True))
                myfile.write('\r\n')
                # 產生要發佈到Kafka的訊息
                future1 = producer1.send(topic="kibana_test", value=user_event)
                record_metadata1 = future1.get(timeout=10)
                print("Message sending completed!")
                time.sleep(random.randint(1,5))
        elif query_string_dict.get('folder')[0] == '圖片找鞋':
            line_bot_api.reply_message(
                event.reply_token,
                quickReplyTextSendMessage
            )
            #擷取tag訊息
            dn = "ID_message/user_menu_event.txt"
            user_event = {}
            with open(dn, "a") as myfile:
                user_event["user_id"] = vars(user_profile)["user_id"]
                user_event["menu"] = query_string_dict.get('tag')[0]
                user_event["time"] = event.timestamp
                myfile.write(json.dumps(user_event,sort_keys=True))
                myfile.write('\r\n')
                # 產生要發佈到Kafka的訊息
                future1 = producer1.send(topic="kibana_test", value=user_event)
                record_metadata1 = future1.get(timeout=10)
                print("Message sending completed!")
                time.sleep(random.randint(1,5))
        else:
            #初始使用者訊息
            gender_event = ["male", "womale"]
            age_event = ["11-20", "21-30", "31-40", "41-50"]
            career_event = ["Freelance", "services", "financial_and_securities", "manufacturing", "electronics", "state-ownedorpublic_education", "students", "information_industry", "home_management", "medical_staff", "other_industries"]
            database = "TEST2"
            lineuserid = vars(user_profile)["user_id"]
            userid = line_user_id_change_to_userid(database, lineuserid)
            dc = "ID_message/user_ID_eventt.json"
            with open(dc, "w") as myfile:
                user_ID_event = {"uesrID"=userid, "age"=null, "gender"=null, "career"=null}
                print(user_ID_event)
                json.dump(user_ID_event, myfile)
            
            if query_string_dict.get('tag')[0] in gender_event:
                line_bot_api.reply_message(event.reply_token, result_message_array)
                #擷取tag訊息
                dn = "ID_message/user_gender_event.txt"
                user_event = {}
                with open(dn, "a") as myfile:
                    user_event["user_id"] = vars(user_profile)["user_id"]
                    user_event["gender"] = query_string_dict.get('tag')[0]
                    user_event["time"] = event.timestamp
                    myfile.write(json.dumps(user_event,sort_keys=True))
                    myfile.write('\r\n')
                    
                dc = "ID_message/user_ID_eventt.json"
                with open(dc, "r") as myfile:
                    user_ID_event = json.load(myfile)
                with open(dc, "w") as myfile:
                    user_ID_event["gender"] = query_string_dict.get('tag')[0]
                    print(user_ID_event)
                    json.dump(user_ID_event, myfile)
                    
            elif query_string_dict.get('tag')[0] in age_event:
                line_bot_api.reply_message(event.reply_token, result_message_array)
                #擷取tag訊息
                dn = "ID_message/user_age_event.txt"
                user_event = {}
                with open(dn, "a") as myfile:
                    user_event["user_id"] = vars(user_profile)["user_id"]
                    user_event["age"] = query_string_dict.get('tag')[0]
                    user_event["time"] = event.timestamp
                    myfile.write(json.dumps(user_event,sort_keys=True))
                    myfile.write('\r\n')
                
                dc = "ID_message/user_ID_eventt.json"
                with open(dc, "r") as myfile:
                    user_ID_event = json.load(myfile)
                with open(dc, "w") as myfile:
                    user_ID_event["gender"] = query_string_dict.get('tag')[0]
                    print(user_ID_event)
                    json.dump(user_ID_event, myfile)
                    
            elif query_string_dict.get('tag')[0] in career_event:
                line_bot_api.reply_message(event.reply_token, result_message_array)
                #擷取tag訊息
                dn = "ID_message/user_career_event.txt"
                user_event = {}
                with open(dn, "a") as myfile:
                    user_event["user_id"] = vars(user_profile)["user_id"]
                    user_event["footwear"] = query_string_dict.get('tag')[0]
                    user_event["time"] = event.timestamp
                    myfile.write(json.dumps(user_event,sort_keys=True))
                    myfile.write('\r\n')
                
                dc = "ID_message/user_ID_eventt.json"
                with open(dc, "r") as myfile:
                    user_ID_event = json.load(myfile)
                with open(dc, "w") as myfile:
                    user_ID_event["gender"] = query_string_dict.get('tag')[0]
                    print(user_ID_event)
                    data = user_ID_event
                    ID = insert_data_to_mysql_database_table_users(data, database)
                    print(ID)
                    json.dump(user_ID_event, myfile)
#                     # 產生要發佈到Kafka的訊息
#                     future1 = producer1.send(topic="kibana_test", value=user_ID_event)
#                     record_metadata1 = future1.get(timeout=10)
#                     print("Message sending completed!")
#                     time.sleep(random.randint(1,5))
            else:
            ###############################
                ##發送訊息
                line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array)
        
    elif 'menu' in query_string_dict:
        #讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
  
        #發送訊息
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        ) 
        #擷取tag訊息
        dn = "ID_message/user_menu_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["menu"] = query_string_dict.get('tag')[0]
            user_event["time"] = event.timestamp
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
            # 產生要發佈到Kafka的訊息
            future1 = producer1.send(topic="kibana_test", value=user_event)
            record_metadata1 = future1.get(timeout=10)
            print("Message sending completed!")
            time.sleep(random.randint(1,5))
    elif 'tag' in query_string_dict:

        
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = event.timestamp
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
            # 產生要發佈到Kafka的訊息
            future1 = producer1.send(topic="kibana_test", value=user_event)
            record_metadata1 = future1.get(timeout=10)
            print("Message sending completed!")
            time.sleep(random.randint(1,5))
            


## 圖片訊息

In [9]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取照片的主顏色

'''
#!pip install clarifai
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os
import pymysql
from urllib.parse import parse_qs


@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
        
    message_content = line_bot_api.get_message_content(event.message.id)
    user_profile = line_bot_api.get_profile(event.source.user_id)    

    #擷取照片的主顏色
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

     #同時存在一個，上限五則訊息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text='Image has Upload'+ ' ' + event.message.id),
         TextSendMessage(text='等待30秒後，請稍後再做確認')
        ]
    )        


    #路徑為相對路徑
    Image_shoes = './images/'+event.message.id+'.jpg'

    pwd = '/root/darknet/result_img/'
    for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
        filepath = pwd + img
        if os.path.exists(filepath):
            os.remove(filepath)
    pwd = '/root/linechatbot/素材/'
    for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
        filepath = pwd + img
        if os.path.exists(filepath):
            os.remove(filepath)

    # 送入darknet檢測照片
    CopyFileWaitToTest(Image_shoes)
    test_pic()
    os.chdir("/root/linechatbot")
    
    

    # ========================== 載入顏色模型，並開起 MySQL select 顏色 回傳 推薦前 6筆商品資料 並將商品資料丟入輪播系統 =========
    
    
    
    
    

#     input_list = []
#     random.shuffle(input_list)
#     proutID = (i for i in input_list)
#     # 連接 MySQL 資料庫
#     db = pymysql.connect(host="chatbot_db",
#         user="root", passwd="123456", db="")
#     cursor = db.cursor()

#     # 執行 MySQL 查詢指令
#     cursor.execute("SELECT col0 col1 col2 col3 col4 col5 FROM db_table where col0 IN userid;")

#     # 取回所有查詢結果
#     results = cursor.fetchall()

#     # 輸出結果
#     num=len(results)
#     for col0, col1, col2, col3, col4, col5 in results:
#         list_shoes=[[col0, col1, col2, col3, col4, col5]*num for i in range (num)]

#     # 關閉連線
#     db.close()
#     print(list_shoes)
#     number = len(list_shoes)
#     print(number)
#     for i in range(0, number):
#         #輸入值
#         web_shoes = list_shoes[i][1]
#         web_pic = list_shoes[i][2]
#         main_color = list_shoes[i][0]
#         second_color = list_shoes[i][3]
#         name_shoes = list_shoes[i][4]

#         #循環加到 list_col_color
#         col_1 = {
#                 "thumbnailImageUrl": web_pic,
#                 "title": "所顏是否?",
#                 "text": main_color,
#                 "actions": [
#                   {
#                     "type": "postback",
#                     "label": second_color,
#                     "text": web_shoes,
#                     "data": "tag=maincolor|Secondarycolor&product_ID="+name_shoes
#                   }
#                 ]
#               }
#         list_col_color.append(col_1)
#     print(list_col_color)

#     #輸出到linebot輪播
#     dc = '素材/商品推薦/reply.json'
#     data =[
#         {
#         "type": "template",
#         "altText": "this is a carousel template",
#         "template":
#         {
#             "type": "carousel",
#             "actions": [],
#             "columns": list_col_color
#         }
#         }
#     ]
    # ======================================================================================================================
    
    
    
    
    

    

In [10]:
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp

def GetColor(filePath):
    # ======================= 官方固定格式 ===============================
    app_color = ClarifaiApp(api_key='413ff636846a4b82a455dd0041100ec8')
    # model = app.public_models.general_model
    model = app_color.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=filePath)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)
    return color_data

In [11]:

import os
import shutil

def CopyFileWaitToTest(Image_shoes):
    Image_shoes = Image_shoes.split("/")[-1]
    Image_shoes = "/root/linechatbot/images/" + Image_shoes
    des_pwd = '/root/darknet/data/'
    fn = 'test.jpg'
    shutil.copyfile(Image_shoes, des_pwd+fn)


In [12]:

import os

def test_pic():   
    os.chdir("/root/darknet")
    cmd ="""
        ./darknet detector test \
        shoe_person/cfg/obj.data \
        shoe_person/cfg/yolov3.cfg \
        shoe_person/cfg/weights/yolov3_last.weights -thresh 0.5\
        data/test.jpg \
        -dont_show &
        """
    os.popen(cmd)


In [13]:
import os
import shutil

def CopyFileWaitToPost(user_id, messageID):
    origin_pwd = '/root/darknet/result_img/'
    des_pwd = '/root/linechatbot/素材/user/'+user_id+'/'
    if not os.path.exists(des_pwd):
        os.makedirs(des_pwd)
        
    # 清空原先測試照片
    for img in [i for i in os.listdir(des_pwd) if '.jpg' in i]:
        os.remove(des_pwd+img)
    img_list = os.listdir(origin_pwd)
    returnImgList = []
    for i in img_list:
        if 'shoe' in i:
            print(i)
            NewfileName = messageID + '-' + i
            shutil.copyfile(origin_pwd+i, des_pwd+NewfileName)
            returnImgList.append(NewfileName)
    return returnImgList

In [14]:
import pymysql
def from_mysql_shoesID_to_line_chat_bot_more(database, prouct_ID):
# database = "TEST2"
# prouct_ID=[('11040',), ('19228',), ('100007_BKW',), ('1011A042-403',), ('1011A131-031',), ('1011A158-100',), ('1011A169-402',), ('1011A176-402',), ('1011A257-001',), ('1011A538-400',), ('1011A540-400',), ('1011A607-001',), ('1011A633-020',), ('1011A690-020',), ('1011A708-001',), ('1012A002-401',), ('1012A117-002',), ('1012A145-020',), ('1012A150-001',), ('1012A155-400',), ('1012A156-022',), ('1012A156-500',), ('1012A159-401',), ('1012A461-401',), ('1012A469-020',), ('1012A500-403',), ('1012A591-002',), ('1012A757-001',), ('1021A032-001',), ('1021A065-005',), ('1021A104-024',), ('1021A104-100',), ('1021A148-001',), ('1021A153-001',), ('1021A153-021',), ('1021A159-001',), ('1021A172-001',)]
    prouct=[tuple(i)[0] for i in prouct_ID]
#     print(prouct)
    shoesID_list = []
    for input_data in prouct:
        db = pymysql.connect(host="chatbot_db",user="root", passwd="123456", db=str(database))#"TEST2"
        cursor = db.cursor()
        cursor.execute("SELECT pid,cata_label,url,mcolor_name,scolor_name,pic_url,brand,pro_name FROM `shoes_data1` WHERE pid IN ('"+input_data+"')")
        mysql_input_data = [i for i in cursor]
#         print(mysql_input_data)
        shoesID_list.append(mysql_input_data)
#         print(shoesID_list)
    shoesID=[x for x in shoesID_list if x]
#     print(shoesID)
    return shoesID

In [15]:
import pymysql
def from_mysql_table_userId_rating_to_shoesID_list(database, userId):
    db = pymysql.connect(host="chatbot_db",user="root", passwd="123456", db=str(database))#"TEST2"
    cursor = db.cursor()
    cursor.execute("SELECT shoesIDX FROM `userId_rating` WHERE userId IN ("+str(userId)+")")
    shoesIDX_tuple = [i for i in cursor]
    #     print(shoesIDX)
    shoesID_list = []
    for input_data in shoesIDX_tuple:
        db = pymysql.connect(host="chatbot_db",user="root", passwd="123456", db=str(database))#"TEST2"
        cursor = db.cursor()
        cursor.execute("SELECT shoesID FROM `shoes_IDX` WHERE shoesIDX IN ("+str(input_data[0])+")")
        for x in cursor:
            shoesID_list.append(x)
#     print(shoesID_list)
    return shoesID_list

In [ ]:
def insert_data_to_mysql_database_table_users(data, database)
#     data = ('612', '11-20', 'male', 'sportsman')
#     database = "TEST2"
    db = mysql.connector.connect(host="chatbot_db",user="root", passwd="123456", db=database)
    cursor = db.cursor()
    sql = "INSERT INTO users (uesrID, age, gender, career) VALUES (%s, %s, %s, %s)"
    val = data # tuple
    cursor.execute(sql, val)
    db.commit()
    print("1 record inserted, ID:", data[0])
    return data[0]

In [ ]:
def line_user_id_change_to_userid(database, lineuserid):
#     database = "TEST2"
#     lineuserid = "U07eb38fc5e2318fee7f2ff8cf7cdad9b"
    db = mysql.connector.connect(host="chatbot_db",user="root", passwd="123456", db=database)
    cursor = db.cursor()
    #先判斷lineuserid = "U07eb38fc5e2318fee7f2ff8cf7cdad6g"是否在表lineuserid_to_users中
    cursor.execute("SELECT lineuserid FROM `lineuserid_to_users`")
    lineuserid_list = [i[0] for i in cursor]
    # print(lineuserid_list)
    #在表中，取　lineuserid　相對　uesrID　的值
    if lineuserid in lineuserid_list:
        cursor.execute("SELECT uesrID FROM `lineuserid_to_users` WHERE lineuserid IN ('"+lineuserid+"')")
        userid_1 = [i[0] for i in cursor]
    #     print(userid_1)
        userid=userid_1[0]
        print(userid)    
    #不在表中
    else:
    # 從lineuserid_to_users表中，確認colume=userId的最終值
        cursor.execute("SELECT uesrID FROM `lineuserid_to_users`")
        userid_list = [i[0] for i in cursor]
        userid_list_1 = list(dict.fromkeys(userid_list))
    #     print(userid_list_1)
        # 輸出userId的最終值＋１
        userid2 = list(map(int, userid_list_1))
    #     print(userid2)
        userid = max(userid2)+1
    #     print(userid)   
        # 新增lineuserid, uesrID　到　lineuserid_to_users表
        sql = "INSERT INTO lineuserid_to_users (lineuserid, uesrID) VALUES (%s, %s)"
        val = (lineuserid, userid) 
        cursor.execute(sql, val)
        db.commit()
        print(userid)
    return userid

In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
[2019-11-05 14:36:08,047] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/root/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/root/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/root/miniconda3/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/root/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/root/miniconda3/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-in

In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])